In [3]:
class Food(object):
    
    def __init__(self, n, v, w):
        self.name = n
        self.value = v
        self.calories = w     
    
    def getValue(self):
        return self.value
    
    def getCost(self):
        return self.calories
    
    def density(self):
        return self.getValue()/self.getCost()
    
    def __str__(self):
        return self.name + ': <' + str(self.value)\
                + ', ' + str(self.calories) + '>'


# Implementing Flexible Greedy

def greedy(items, maxCost, keyFunction):
    """Assumes Items a list, maxWeight >= 0,
        keyFunction maps elements of Items to numbers"""

    itemsCopy = sorted(items, key=keyFunction, reverse = True)

    result = []
    totalValue, totalCost = 0.0, 0.0

    for i in range(len(itemsCopy)):
        if(totalCost + itemsCopy[i].getCost()) <= maxCost:
            result.append(itemsCopy[i])
            totalCost += itemsCopy[i].getCost()
            totalValue += itemsCopy[i].getValue()

    return (result, totalValue)


# Building menu of foods

def buildMenu(names, values, calories):
    """names, values, calories list of same length.
        name a list of strings vlaues and calories lists of 
        numbers.  Returns list of foods"""
    
    #names = ['wine', 'beer', 'pizza', 'burger', 'fries', 
    #           'cola', 'apple', 'donut', 'cake']
    #values = [89, 90, 95, 100, 90, 79, 50, 10]
    #calories = [123, 154, 258, 354, 365, 150, 95, 195]
    #foods = buildMenu(names, values, calories)
    
    menu = []
    for i in range(len(values)):
        menu.append(Food(names[i], values[i], calories[i]))

    return menu

    
def testGreedy(items, constraint, keyFunction):
    taken, val = greedy(items, constraint, keyFunction)
    print('Total value of items taken = ', val)

    for item in taken:
        print(' ', item)
            

def testGreedys(foods, maxUnits):
    print('Use greedy by value to allocate', maxUnits, 
          'calories')
    testGreedy(foods, maxUnits, Food.getValue)
    print('\nUse greedy by cost to allocate', maxUnits, 
          'calories')
    testGreedy(foods, maxUnits,
                lambda x: 1/Food.getCost(x))
    print('\nUse greedy by density to allocate', maxUnits, 
          'calories')
    testGreedy(foods, maxUnits, Food.density)
    
        
#calling:

names = ['wine', 'beer', 'pizza', 'burger', 'fries', 
             'cola', 'apple', 'donut', 'cake']
values = [89, 90, 95, 100, 90, 79, 50, 10]
calories = [123, 154, 258, 354, 365, 150, 95, 195]    
foods = buildMenu(names, values, calories)

#testGreedys(foods, 750)


# ******************* BRUTE FORCE ALGORITHM (start here)***************
#                     ----------------------

# *** Header for Decision Tree Implementation *** 
#----------------------------------------------------------

def maxVal(toConsider, avail):
    """Assumes toConsider a list of items, avail a weight
       Returns a tuple of the total value of a solution to the 
       0/1 knapsack problem and the items of that solution"""
    # toConsider--> items that nodes higher up the tree, not yet considered
    # avail --> the amount of space "still available"
    
    if toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getCost() > avail:
        # explore right branch only
        result = maxVal(toConsider[1:], avail)
    else:
        nextItem = toConsider[0]
        # explore left branch
        withVal, withToTake = maxVal(toConsider[1:], 
                                     avail - nextItem.getCost())
        withVal += nextItem.getValue()
        # explore right branch
        withoutVal, withoutToTake = maxVal(toConsider[1:], avail)
        # choose better branch
        if withVal > withoutVal:
            reuslt = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    return result
        
# Doesn't actually build search tree
# Local variable "result" records best solution found so far

import random

def buildLargeMenu(numItems, maxVal, maxCost):
    items = []
    for i in range(numItems):
        items.append(Food(str(i),
                         random.randint(1, maxVal),
                         random.randint(1, maxCost)))
    return items

#for numItems in (5, 10, 15, 20, 25, 30, 35, 40, 45, 50):
#    print('Try a menu with', numItems, 'items')
#    items = buildLargeMenu(numItems, 90, 250)
#    testMaxVal(items, 750, fastMaxVal, False)

#    testMaxVal(items, 750, False)


def fastMaxVal(toConsider, avail, memo = {}):
    """Assumes toConsider a list of items, avail a weight
       memo supplied by recursive calls
       Returns a tuple of the total value of a solution to the 
       0/1 knapsack problem and the items of that solution"""
    
    if (len(toConsider), avail) in memo:
        result = memo[(len(toConsider), avail)]
    elif toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getCost() > avail:
        # exploreing right branch only
        result = fastMaxVal(toConsider[1:], avail, memo)
    else:
        nextItem = toConsider[0]
        # Explore left branch
        withVal, withToTake =\
                fastMaxVal(toConsider[1:],
                          avail - nextItem.getCost(), memo)
        withVal += nextItem.getValue()
        # Explore right branch
        withoutVal, withoutToTake = fastMaxVal(toConsider[1:], avail, memo)
        
        # Choose better branch
        if withVal > withoutVal:
            result = (withVal, withoutToTake + (nextItem, ))
        else:
            result = (withoutVal, withoutToTake)
    memo[(len(toConsider), avail)] = result
    
    return result


def testMaxVal(foods, maxUnits, algorithm, printItems = True):
    print('Menu contains', len(foods), 'items')
    print('Use search tree to allocate', maxUnits, 
          'calories')
    val, taken = algorithm(foods, maxUnits)
    if printItems:
        print('Total value of items taken = ', val)
        for item in taken:
            print('  ', item)
            

            
#Calling
for numItems in (5, 10, 15, 20, 25, 30, 35, 40, 45, 50):
    print('Try a menu with', numItems, 'items')
    items = buildLargeMenu(numItems, 90, 250)
    
    testMaxVal(items, 750, fastMaxVal, False)
    


Use greedy by value to allocate 750 calories
Total value of items taken =  284.0
  burger: <100, 354>
  pizza: <95, 258>
  wine: <89, 123>

Use greedy by cost to allocate 750 calories
Total value of items taken =  318.0
  apple: <50, 95>
  wine: <89, 123>
  cola: <79, 150>
  beer: <90, 154>
  donut: <10, 195>

Use greedy by density to allocate 750 calories
Total value of items taken =  318.0
  wine: <89, 123>
  beer: <90, 154>
  cola: <79, 150>
  apple: <50, 95>
  donut: <10, 195>
